# Visualización de Datos con Python

## Proyecto Final: Creación de un Tablero de Datos Interactivo con Dash

El objetivo de este proyecto es desarrollar un tablero de datos interactivo utilizando la biblioteca [Dash](https://dash.plotly.com/) en Python. Para ello se pone a disposición un conjunto de datos histórico sobre los Juegos Olímpicos modernos para construir visualizaciones significativas y permitir interacciones con el usuario.


## 1. Carga de datos y armado del Dataset.

In [1]:
# Instalar los paquetes necesarios
%%capture
!pip install plotly dash

In [2]:
%%capture
!pip install dash-bootstrap-components plotly dash jupyter-dash

In [3]:
# Importar las bibliotecas necesarias
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import dash_bootstrap_components as dbc
from dash import Dash, dcc, html, Input, Output

In [4]:
# Importamos el DataFrame
df = pd.read_csv("juegos_olimpicos.csv")

# Se eliminan los registros duplicados.
# Se consideran los duplicados en todas las columnas (subset=None)
# Se selecciona la primer ocurrencia con keep="first".
# Se pisa el dataframe original con inplace.

df.drop_duplicates(subset=None, keep="first", inplace =True)

# Visualización

In [5]:
# Definición del estilo del lienzo
stylesheet = [dbc.themes.BOOTSTRAP]

# Definición de la app
app = Dash(__name__, external_stylesheets=stylesheet, suppress_callback_exceptions=True)

# Definición del layout como propiedad de la app
app.layout = html.Div([html.H1("Tablero Interactivo de los Juegos Olímpicos (1896-2016)"),
    dcc.Tabs(id="tabs", value="tab_1", children=[
        dcc.Tab(label="Conteo de Medallas", value="tab_1"),
        dcc.Tab(label="Participación por Sexo", value="tab_2"),
        dcc.Tab(label="Representación Geográfica", value="tab_3"),
        dcc.Tab(label="Distribución de Edad y Sexo", value="tab_4")]),
    html.Div(id="tabs-content")])

# Generación de renderizado
@app.callback(Output("tabs-content", "children"),
              [Input("tabs", "value")])
def render_content(tab):

    if tab == "tab_1":
      return html.Div([
          html.H3("Conteo de Medallas"),
          html.H6("Selector de cantidad de países"),
          dcc.Dropdown(id="k_selector_ej_1", placeholder="Seleccionar cantidad de países a mostrar"),
          html.Br(),
          dcc.Checklist(id="medal_selector_ej_1",options=[
              {"label": html.Div(['Gold'], style={'color': 'Gold', 'font-size': 20}), "value": "Gold"},
               {"label": html.Div(['Silver'], style={'color': 'Silver', 'font-size': 20}), "value": "Silver"},
                {"label": html.Div(['Bronze'], style={'color': 'brown', 'font-size': 20}), "value": "Bronze"},], value=["Gold", "Silver", "Bronze"], labelStyle={"display": "flex", "align-items": "center"}),
          html.Br(),
          html.H6("Selector de disciplina"),
          dcc.Dropdown(id="sport_selector_ej_1", placeholder="Seleccionar la disciplina a mostrar"),
          html.Br(),
          dcc.RangeSlider(id="year_slider_ej_1", min=df["Year"].min(), max=df["Year"].max(), step=1, value=[df["Year"].min(), df["Year"].max()], allowCross=False, marks={str(year): str(year) for year in df["Year"].unique()},),
          html.Br(),
          dcc.Graph(id="fig_ej_1")])

    elif tab == "tab_2":
        return html.Div([
            html.H3("Participación por Sexo"),
            dcc.RangeSlider(id="year_slider_ej_2", min=df["Year"].min(), max=df["Year"].max(), step=1, value=[df["Year"].min(), df["Year"].max()], marks={str(year): str(year) for year in df["Year"].unique()},),
            dcc.Graph(id="fig_ej_2")])

    elif tab == "tab_3":
          return html.Div([
              html.H3("Representación Geográfica"),
              dcc.Dropdown(id="year_selector_ej_3", options=[], value=None),
              dcc.Graph(id="fig_ej_3")])

    elif tab == "tab_4":
      return dbc.Container(
    html.Div([
        html.H3("Distribución de Edad por Sexo y Disciplina"),

        # Contenedor para los controles
        dbc.Row(
            dbc.Col(
                dbc.Card(
                    dbc.CardBody([
                        html.H6("Selección de disciplina"),
                        dcc.Dropdown(id="sport_selector_ej_4", options=[], placeholder="Seleccionar disciplina a mostrar"),
                        html.Br(),
                        html.H6("Rango de Olimpiadas"),
                        dcc.RangeSlider(id="year_slider_ej_4", min=df["Year"].min(), max=df["Year"].max(), step=1, value=[df["Year"].min(), df["Year"].max()], allowCross=False,
                                        marks={str(year): str(int(year)) for year in df["Year"].unique()}),]),
                    style={"padding": "20px", "border": "2px solid black", "border-radius": "10px"}), width=12), justify="center", style={"margin-bottom": "20px"}),

        # Contenedor para los gráficos y controles de edad
        dbc.Row([
            dbc.Col(dcc.Graph(id="fig_ej_4"), width=8),  # Histograma
            dbc.Col(
                html.Div([
                    dbc.Card(
                        dbc.CardBody([
                            html.H6("Seleccionar Edad para el Gráfico de Torta"),
                            dcc.Dropdown(id="age_dropdown_ej_4", placeholder="Seleccionar edad", style={"width": "100%"})]),
                        color="light", outline=True, style={"padding": "20px", "border": "2px solid black", "border-radius": "10px"}),
                    dcc.Graph(id="pie_chart_ej_4")  # Gráfico de torta
                ]), width=4)], justify="center", align="center")]),
    fluid=True)



# Definición de callback tab_1
@app.callback([Output("k_selector_ej_1", "options"),
               Output("sport_selector_ej_1", "options"),
               Output("fig_ej_1", "figure")],
                [Input("k_selector_ej_1", "value"),
                 Input("sport_selector_ej_1", "value"),
                 Input("medal_selector_ej_1", "value"),
                 Input("year_slider_ej_1", "value")])

def update_ejercicio_1(selected_k, selected_sport, selected_medals, selected_year_range):

    # Filtrar por rango de años y medallas
    df_filtered = df[(df["Year"] >= selected_year_range[0]) & (df["Year"] <= selected_year_range[1])]
    df_filtered = df_filtered[df_filtered["Medal"].isin(selected_medals)]

    # Opciones de cantidad de países
    k_options = [{"label": str(i), "value": i} for i in range(1, len(df_filtered["NOC"].unique()) + 1)]

    # Opciones de disciplinas
    sport_options = [{"label": "all sports", "value": "all sports"}] + \
                    [{"label": sport, "value": sport} for sport in sorted(df_filtered["Sport"].unique())]

    # Verificación de entradas válidas
    if not selected_k or not selected_sport:
      fig = px.bar(title="Por favor, seleccione todas las opciones", labels={"x": "Medalla", "y": "Número de Medallas"})
      return k_options, sport_options, fig

    # Filtrado adicional por deporte
    df_filtered_ej_1 = df_filtered[df_filtered["Sport"] == selected_sport] if selected_sport != "all sports" else df_filtered

    # Verificación de datos después del filtrado
    if len(df_filtered_ej_1) == 0:
      fig = px.bar(title="No hay datos para mostrar con los filtros seleccionados", labels={"x": "Medalla", "y": "Número de Medallas"})
      return k_options, sport_options, fig

    # Agrupamiento por país y medalla
    df_grouped = df_filtered_ej_1.groupby(["NOC", "Medal"]).size().unstack(fill_value=0).reset_index()

    # Verificación de datos después del agrupamiento
    if len(df_grouped) == 0:
        fig = px.bar(title="No hay datos para mostrar con los filtros seleccionados", labels={"x": "Medalla", "y": "Número de Medallas"})
        return k_options, sport_options, fig

    # Verificar medallas disponibles
    available_medals = [medal for medal in selected_medals if medal in df_grouped.columns]
    if not available_medals:
        fig = px.bar(title="No hay medallas disponibles para los filtros seleccionados", labels={"x": "Medalla", "y": "Número de Medallas"})
        return k_options, sport_options, fig

    # Ordenar y seleccionar los top k países
    df_grouped["total_medals"] = df_grouped[available_medals].sum(axis=1)
    df_grouped = df_grouped.sort_values(by="total_medals", ascending=False)
    selected_k = min(selected_k, len(df_grouped))
    df_top_paises = df_grouped.head(selected_k)

    # Transformar DataFrame a formato largo
    df_long = pd.melt(df_top_paises, id_vars=['NOC'], value_vars=available_medals, var_name='Medal', value_name='Number of Medals')

    # Crear la figura con Plotly
    fig = px.bar(df_long, x='Number of Medals', y='NOC', color='Medal', barmode='group', color_discrete_map={"Gold": "gold", "Silver": "silver", "Bronze": "brown"}, category_orders={"NOC": df_top_paises['NOC'].tolist()},
                 title=f"Top {selected_k} países con más medallas en {selected_sport if selected_sport != 'all sports' else 'todos los deportes'}")
    fig.update_layout(xaxis_title="Número de Medallas", yaxis_title="Países")

    return k_options, sport_options, fig


# Definición de callback tab_2
@app.callback(Output("fig_ej_2", "figure"),
 [Input("year_slider_ej_2", "value")])

def update_ejercicio_2(selected_year_range):
    # Filtramos el dataset por el rango de años seleccionado
    df_anios = df[(df["Year"] >= selected_year_range[0]) & (df["Year"] <= selected_year_range[1])]

    # Agrupamos y contamos número de varones y mujeres por país y año
    df_agrupado = df_anios.groupby(["NOC", "Year", "Sex"]).size().reset_index(name="count")

    # Pivoteamos la tabla para tener columnas separadas para varones y mujeres
    df_sexo = df_agrupado.pivot(index=["NOC", "Year"], columns="Sex", values="count").fillna(0).reset_index()

    # Convertimos la columna "Year" a string para tratarla como categoría y mejorar la visualización
    df_sexo["Year"] = df_sexo["Year"].astype(str)

    # Creamos un scatter plot con Plotly Express, diferenciado un color para cada año
    fig = px.scatter(df_sexo, x="M", y="F", color="Year", labels={"M": "Cantidad de Hombres", "F": "Cantidad de Mujeres"},
                     title="Participación por Sexo en los Juegos Olímpicos, según país y año",
                     color_discrete_sequence=px.colors.qualitative.Bold)

    # Añadimos regresiones lineales para cada año, con color degradado en la línea
    years = df_sexo["Year"].unique()
    color_scale = px.colors.sequential.Viridis

    for i, year in enumerate(years):
        df_year = df_sexo[df_sexo["Year"] == year]
        x = df_year["M"]
        y = df_year["F"]

    # Calculamos la regresión a través de una función lineal -con ayuda de chatgpt-
        coef = np.polyfit(x, y, 1)
        poly1d_fn = np.poly1d(coef)

    # Agregamos la regresión al gráfico con go.Scatter (librería de plotly.graph_objects)
        fig.add_trace(go.Scatter(x=x, y=poly1d_fn(x), mode='lines', name=f'Regresión {year}'))

    return fig


# Definición de callback tab_3
@app.callback(
    [Output("year_selector_ej_3", "options"), Output("year_selector_ej_3", "value"), Output("fig_ej_3", "figure")],
    [Input("year_selector_ej_3", "value")]
)
def update_dropdown_and_figure(selected_year):
    # Obtener y ordenar los años únicos
    years = sorted(df['Year'].unique())
    options = [{"label": str(year), "value": year} for year in years]

    # Establecer el valor predeterminado si no hay un año seleccionado
    if not selected_year or selected_year not in years:
        selected_year = years[0]

    # Filtrar el DataFrame por el año seleccionado
    df_anios_2 = df[df["Year"] == selected_year]

    # Agrupar y contar el número de atletas por país
    df_atletas = df_anios_2.groupby("NOC").size().reset_index(name="count")

    # Crear el planisferio de distribución de atletas por país usando la función choropleth de Plotly
    fig = px.choropleth(
        df_atletas,
        locations="NOC",
        color="count",  # Variable a representar
        hover_name="NOC",  # Mostrar el código del país al señalar en el mapa
        color_continuous_scale="Purples",
        title=f"Distribución Geográfica de Atletas en los Juegos Olímpicos en {selected_year}"
    )

    return options, selected_year, fig


# Definición de callback tab_4
@app.callback([Output("sport_selector_ej_4", "options"),
               Output("age_dropdown_ej_4", "options"),
               Output("fig_ej_4", "figure"),
               Output("pie_chart_ej_4", "figure")],
                [Input("year_slider_ej_4", "value"),
                 Input("sport_selector_ej_4", "value"),
                 Input("age_dropdown_ej_4", "value")])

def update_ejercicio_4(selected_year_range, selected_sport, selected_age):

    # Obtener disciplinas únicas y ordenarlas alfabéticamente
    sports = sorted(df["Sport"].unique())
    lista_sports = [{"label": "all sports", "value": "all sports"}] + [{"label": sport, "value": sport} for sport in sports]

    # Filtrar los datos según el rango de años y deporte
    df_filtered_ej_4 = df[(df["Year"] >= selected_year_range[0]) & (df["Year"] <= selected_year_range[1])]

    if selected_sport != "all sports":
        df_filtered_ej_4 = df_filtered_ej_4[df_filtered_ej_4["Sport"] == selected_sport]

    # Actualización de opciones del Dropdown de edad
    edades_disponibles = sorted(df_filtered_ej_4["Age"].dropna().unique(), reverse=True)
    lista_edad = [{"label": str(int(age)), "value": age} for age in edades_disponibles]

    # Actualización del histograma
    if not selected_year_range or len(selected_year_range) != 2:
        fig_histogram = px.histogram(x=[], title="Rango de olimpiadas seleccionado no válido", labels={"x": "Edad"})
        fig_histogram.update_layout(xaxis_title="Edad", yaxis_title="Número de Atletas", title=dict(text="Rango de olimpiadas seleccionado no válido", font=dict(size=18)))

        fig_pie = px.pie(title="Seleccione una edad válida") # Gráfico de torta vacío

        return lista_sports, lista_edad, fig_histogram, fig_pie


    # Filtrar por deporte
    df_filtered_ej_4 = df_filtered_ej_4[df_filtered_ej_4["Sport"] == selected_sport] if selected_sport != "all sports" else df_filtered_ej_4

    if len(df_filtered_ej_4) == 0:
      fig_histogram = px.histogram(x=[], title="No hay datos para mostrar con los filtros seleccionados", labels={"x": "Edad"})
      fig_histogram.update_layout(xaxis_title="Edad", yaxis_title="Número de Atletas", title=dict(text="No hay datos para mostrar con los filtros seleccionados", font=dict(size=18)))

      fig_pie = px.pie(title="No hay datos para mostrar con los filtros seleccionados") # Gráfico de torta vacío

      return lista_sports, lista_edad, fig_histogram, fig_pie


    # Cambiar etiquetas de género
    df_filtered_ej_4["Sex"] = df_filtered_ej_4["Sex"].replace({"F": "Mujeres", "M": "Hombres"})

    # Crear el histograma
    fig_histogram = px.histogram(df_filtered_ej_4, x="Age", color="Sex", nbins=30,
                                 title=f"Distribución de Edad por Sexo y Disciplina: {selected_sport}",
                                 labels={"Age": "Edad"}, color_discrete_map={"Mujeres": "#FF97FF", "Hombres": "#19D3F3"})
    fig_histogram.update_layout(barmode="overlay", xaxis_title="Edad", yaxis_title="Número de Atletas", plot_bgcolor='rgba(0, 0, 0, 0)',
                                title=dict(text=f"Distribución de Edad por Sexo y Disciplina: {selected_sport}", font=dict(size=18)))
    fig_histogram.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')
    fig_histogram.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')
    fig_histogram.update_traces(opacity=0.6)

    # Para el gráfico de tortas, filtrar para la edad seleccionada y agrupar por Sexo
    df_pie = df_filtered_ej_4[df_filtered_ej_4["Age"] == selected_age]

    if len(df_pie) == 0:
        fig_pie = px.pie(title="No hay datos para la edad seleccionada")
    else:
        df_pie = df_pie.groupby("Sex").size().reset_index(name="Count")

        fig_pie = px.pie(df_pie, names="Sex", values="Count", title=f"Distribución por Género para la Edad {selected_age}",
                         labels={"Sex": "Sexo", "Count": "Número de Atletas"}, color="Sex", color_discrete_map={"Mujeres": "rgb(253,205,172)", "Hombres": "rgb(56,166,165)"})
        fig_pie.update_traces(textinfo="percent+label")
        fig_pie.update_layout(title=dict(text=f"Distribución por Género para el rango <br> de olimpiadas seleccionado y edad de {selected_age}",
                                         font=dict(size=16), x=0.5, xanchor='center', y=0.95, yanchor='top'), title_font_size=16, margin=dict(t=80))

    return lista_sports, lista_edad, fig_histogram, fig_pie


if __name__ == "__main__":
    app.run(debug=True)

<IPython.core.display.Javascript object>